<a href="https://colab.research.google.com/github/darwinyusef/UsaHousingLab/blob/master/LaboratorioActividad2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow --upgrade
!pip install keras
!pip install sklearn
!pip install matplotlib
!pip install seaborn

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
# from keras.models import Model
# from keras.layers import Input, Dense
# from keras.optimizers import Adam
# from sklearn.preprocessing import MinMaxScaler
from scipy import stats

In [ ]:
!wget https://darwinyusef.github.io/UsaHousingLab/creditcardcsvpresent.csv

# Introducción

En este laboratorio se realizó un análisis de segmentación de clientes basado en un dataset relacionado con transacciones de tarjetas de crédito. El objetivo principal fue aplicar técnicas de clustering (agrupamiento no supervisado) con el algoritmo K-Means para identificar patrones o grupos de clientes que presenten comportamientos similares.

Este tipo de análisis es útil en sistemas de detección de fraude, segmentación de marketing o análisis de riesgo crediticio.

Fuente del Dataset
# Dataset: Credit Card Fraud Detection

Abstract Data Set for Credit Card Fraud Detection

Link del CSV utilizado:
https://github.com/darwinyusef/UsaHousingLab/blob/master/creditcardcsvpresent.csv

In [ ]:
# Now read the CSV file
df = pd.read_csv("/content/creditcardcsvpresent.csv", sep=',')

# Display the first few rows
df.head()

In [ ]:
df.info() # actualmente tiene 3075 elementos

## Descripción Detallada del Dataset

El dataset utilizado corresponde a un problema clásico de detección de fraude en transacciones con tarjetas de crédito. Contiene un total de 31 columnas y fue preprocesado para proteger la confidencialidad de los clientes mediante técnicas de anonimización.

### Características Generales:

- Total de filas (instancias de transacciones): 284,807
- Total de columnas (atributos): 31
- La mayoría de las variables han sido transformadas usando Análisis de Componentes Principales (PCA) para preservar la privacidad de los datos reales.

---

### Descripción de las Variables

| Variable | Descripción | Tipo de Dato |
|----------|-------------|--------------|
| Time     | Tiempo en segundos transcurrido desde la primera transacción registrada en el dataset. | Numérico |
| V1 a V28 | Componentes principales generados por PCA sobre las variables originales. No se conoce el significado exacto por razones de confidencialidad. | Numérico |
| Amount   | Monto de la transacción realizada. | Numérico |
| Class    | Variable objetivo que indica el tipo de transacción: 0 = Transacción legítima, 1 = Transacción fraudulenta. | Binario (0 o 1) |

---

### Características Importantes del Dataset:

- Es un dataset desbalanceado:  
  - 492 transacciones fraudulentas (0.17%)  
  - 284,315 transacciones legítimas (99.83%)  

- Variables V1 a V28 permiten captar patrones ocultos de comportamiento transaccional gracias a la técnica de PCA.

- La variable `Amount` puede requerir normalización o estandarización previa a la aplicación de algoritmos de clustering.



In [ ]:
# Estadísticas de las variables numéricas
df.describe()

In [ ]:
# @title # Identificar columnas categóricas.
cat_cols = df.select_dtypes(include=['object']).columns
print(cat_cols)
# Frecuencia de categorías
for col in cat_cols:
    print(f"Columna: {col}")
    print(df[col].value_counts())
    print("-" * 40)

## Análisis Exploratorio: Matriz de Correlación

Antes de aplicar cualquier técnica de detección de outliers o clustering, se realizó un análisis de correlación entre las variables más relevantes del dataset. Esto permite entender la relación entre las características y detectar redundancias o dependencias.

### Observaciones relevantes:

- Las variables **`Daily_chargeback_avg_amt`**, **`6_month_avg_chbk_amt`** y **`6-month_chbk_freq`** presentan una **alta correlación positiva** entre sí (mayor a 0.85). Esto indica que reflejan comportamientos similares, posiblemente relacionados con el historial de devoluciones (chargebacks).
- Variables como **`Total Number of declines/day`** o **`Merchant_id`** muestran **baja correlación** con el resto, indicando independencia, lo que podría ser útil para detectar anomalías específicas de ciertos comercios o hábitos poco frecuentes Yo elimine una de ellas.

**Seleccionar variables representativas** y evitar redundancia  **Reducir imensiones** de forma más efectiva al aplicar **PCA**. **Interpretar los clusters** y validar que variables como los montos promedio o las devoluciones tienen peso real en la agrupación.


In [ ]:
# Matriz de correlación
corr = df.corr(numeric_only=True)

# Mapa de calor
plt.figure(figsize=(12,8))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f")
plt.title("Matriz de Correlación")
plt.show()

En la figura 2 se muestra un mapa de calor y en la figura 3 una matriz de correlaciones. Analiza en detalle estos datos. ¿Qué variables convertiría a categóricas o factor? ¿Qué variables eliminaría? Justifica la respuesta.

![Matris de ](https://raw.githubusercontent.com/darwinyusef/UsaHousingLab/refs/heads/master/corrconfianza.JPG)

La matriz de correlaciones muestra cómo se relacionan entre sí las variables numéricas del dataset. Los valores cercanos a 1 o -1 indican una fuerte correlación, mientras que los cercanos a 0 indican poca o ninguna relación lineal.

Merchant_id no tiene correlación significativa con otras variables.

Transaction_amount está moderadamente correlacionada con Average.Amount.transaction.day.

Las variables relacionadas con "chargeback" (Daily_chargeback_avg_amt, X6_month_avg_chbk_amt, X6.month_chbk_freq) están fuertemente correlacionadas entre sí (cerca de 0.9).

Total.Number.of.declines.day no muestra una correlación fuerte con otras variables.

## 2. Variables que se pueden convertir a categóricas (factores)

| Variable                     | ¿Convertir a categórica? | Justificación                                                                 |
|-----------------------------|---------------------------|-------------------------------------------------------------------------------|
| `Merchant_id`               | ✅ Sí                     | Es un identificador, no tiene sentido como variable numérica.                |
| `Total.Number.of.declines.day` | ⚠️ Tal vez              | Si los valores son enteros pequeños, podría tratarse como ordinal.          |
| `X6.month_chbk_freq`        | ⚠️ Tal vez                | Si tiene pocos valores únicos, puede considerarse como categórica.           |


## 3. Variables que se pueden eliminar

| Variable                     | ¿Eliminar? | Justificación                                                                 |
|-----------------------------|------------|-------------------------------------------------------------------------------|
| `Merchant_id`               | ✅ Sí      | No tiene valor analítico en análisis numérico; solo es útil como identificador. |
| `Daily_chargeback_avg_amt` | ⚠️ Potencial | Alta correlación con `X6_month_avg_chbk_amt`; puede causar redundancia.      |
| `X6_month_avg_chbk_amt`     | ⚠️ Potencial | Mismo motivo que arriba; mantener solo una de las dos.                       |
| `Transaction_amount` o `Average.Amount.transaction.day` | ❌ No | Aunque correlacionadas, aportan información complementaria.                  |

Yo literal mate a Transaction_amount y Merchant_id

# Detección de Anomalías


La detección de anomalías, también conocida como detección de outliers, es una técnica fundamental en el análisis de datos. Permite identificar comportamientos atípicos o sospechosos que se desvían significativamente del patrón general. En el contexto de los datos financieros, como las transacciones con tarjetas de crédito, detectar estas anomalías puede ayudar a prevenir fraudes o errores operacionales.

Para este estudio, se aplicaron diversas técnicas de clustering y detección de anomalías, tanto con reducción de dimensionalidad (PCA) como sin ella, y se evaluó su rendimiento e interpretabilidad.

### 🧭 Enfoque del Análisis

1. **Preprocesamiento de los datos**:
   - Estandarización de variables.
   - Análisis exploratorio para identificar posibles valores extremos.

In [ ]:
# Revisión de valores nulos
df.isnull().sum()

In [ ]:
# @title Esto debido a que tiene muchisimos nulls es el 100% y se recomienda borrarla cuando tiene (más del 40% del total).
df.drop(columns=['Transaction date', 'Merchant_id'], inplace=True)

# Verificamos que se eliminó
df.head()

In [ ]:
# Rellenar valores nulos con la media (opcionalmente la mediana)
for col in df.select_dtypes(include=[np.number]).columns:
    df[col].fillna(df[col].mean(), inplace=True)

for col in cat_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)



2. **Aplicación de técnicas de detección de outliers**:
   - **Isolation Forest** para detectar puntos que se aíslan fácilmente.
   - **Local Outlier Factor (LOF)** para encontrar observaciones con baja densidad local.
   - **Autoencoder** para identificar errores altos de reconstrucción en los datos.


1. ## Isolation Forest
from sklearn.ensemble import IsolationForest

**Descripción**: Algoritmo basado en árboles que aísla las observaciones anómalas. Funciona construyendo árboles de aislamiento aleatorios; los puntos que requieren menos divisiones para aislarse son considerados anomalías.

finalmente es Eficiente en datasets grandes y de alta dimensión. No requiere etiquetas. es muy comun para Detección de fraudes, errores en sensores, valores atípicos en series temporales.

In [ ]:
from sklearn.ensemble import IsolationForest

# Selección de variables numéricas
num_cols = df.select_dtypes(include=[np.number]).columns

# Modelo Isolation Forest
iso = IsolationForest(contamination=0.01, random_state=42)
df['anomaly'] = iso.fit_predict(df[num_cols])

# Interpretación: -1 = Anómalo, 1 = Normal
print(df['anomaly'].value_counts())

# Visualización de anomalías
sns.scatterplot(x='Transaction_amount', y='Average Amount/transaction/day', hue='anomaly', data=df)
plt.title('Anomalías detectadas')
plt.show()

In [ ]:
z_scores = stats.zscore(df[num_cols])
abs_z_scores = np.abs(z_scores)
outliers = (abs_z_scores > 3).any(axis=1)

df['anomaly_z'] = np.where(outliers, -1, 1)
df['anomaly_z']


In [ ]:
# Normaliza los datos
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df[num_cols])

# Definición del Autoencoder
input_dim = df_scaled.shape[1]
input_layer = Input(shape=(input_dim,))
encoder = Dense(8, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="linear")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

# Compilar
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

# Entrenar solo con datos normales (sin outliers)
autoencoder.fit(df_scaled, df_scaled, epochs=50, batch_size=32, verbose=1)


2. ## Local Outlier Factor (LOF)

from sklearn.neighbors import LocalOutlierFactor

**Descripción**: Evalúa la anomalía de cada muestra comparando su densidad local con la de sus vecinos. Una muestra se considera anómala si tiene una densidad significativamente menor que la de sus vecinos.

Captura relaciones locales; útil cuando las anomalías no son globales pero sí en ciertas regiones del espacio de características.

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

lof = LocalOutlierFactor(n_neighbors=20, contamination=0.01)
df['anomaly_lof'] = lof.fit_predict(df[num_cols])
print(df['anomaly_lof'])
sns.scatterplot(x='Transaction_amount', y='Average Amount/transaction/day', hue='anomaly_lof', data=df)
plt.title('Anomalías LOF')
plt.show()

3. ## Autoencoder  (Red Neuronal no supervisada)

**Descripción**: Red neuronal no supervisada que aprende a comprimir (codificar) y reconstruir (decodificar) los datos. Las muestras que no pueden ser reconstruidas correctamente se consideran anómalas.

Ventajas: Muy potente para datos de alta dimensión. Puede capturar patrones complejos no lineales.

Métrica clave: Se mide el error de reconstrucción; si este error es mayor a un umbral, la instancia se considera una anomalía.

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense

input_dim = df[num_cols].shape[1]
input_layer = Input(shape=(input_dim,))
encoder = Dense(8, activation="relu")(input_layer)
decoder = Dense(input_dim, activation="linear")(encoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)
reconstructions = autoencoder.predict(df_scaled)

# Error cuadrático medio por fila
mse = np.mean(np.power(df_scaled - reconstructions, 2), axis=1)

# Establecer un umbral para considerar anomalías
threshold = np.percentile(mse, 99)  # Top 1% como outliers

df['anomaly_autoencoder'] = np.where(mse > threshold, -1, 1)

print(df['anomaly_autoencoder'].value_counts())


sns.scatterplot(x='Transaction_amount', y='Average Amount/transaction/day', hue='anomaly_autoencoder', data=df)
plt.title('Anomalías detectadas con Autoencoder')
plt.show()


Técnicas de Clustering Utilizadas

| **Técnica**                  | **Entrada**          | **Visualización** | **Observaciones clave**                        |
| ---------------------------- | -------------------- | ----------------- | ---------------------------------------------- |
| **DBSCAN Sin PCA**           | Variables originales | 2D                | Detecta formas arbitrarias y outliers.         |
| **KMeans Con PCA**           | Datos reducidos      | PCA 2D            | Rápido, pero pierde interpretabilidad directa. |
| **Agglomerative Clustering** | Variables originales | Dendrograma       | Muestra jerarquía de agrupación.               |
| **KMeans Sin PCA**           | Variables originales | 2D                | Más interpretable, pero sensible a outliers.   |

---

In [ ]:
# @title ## ---

## Análisis de Clusters con KMeans y Técnicas de Detección de Outliers

En este análisis se utilizó **KMeans**, uno de los algoritmos de clustering más populares, para segmentar los datos de transacciones de tarjetas de crédito. Sin embargo, KMeans por sí solo es **sensible a la presencia de outliers**, ya que estos pueden alterar la posición de los centroides y afectar negativamente la calidad del agrupamiento.

Para mitigar este problema, se aplicaron técnicas de **detección de anomalías previas al clustering**, logrando una mejor separación entre grupos y una identificación más precisa de comportamientos anómalos.

---


3. **Eliminación de outliers detectados** o **etiquetado para análisis paralelo**.

4. **Clustering con KMeans**:
   - Sin reducción de dimensiones: se mantuvo la interpretación directa de las variables.
   - Con PCA: se aplicó reducción de dimensiones para acelerar el entrenamiento y facilitar la visualización.

5. **Visualización de los resultados**:
   - Gráficas 2D de los clusters formados.
   - Comparación de agrupamientos con y sin PCA.
   - Validación del número óptimo de clusters con el **Elbow Method**.

---

### Finalmente explico

- La aplicación de técnicas de outlier **mejoró considerablemente** la coherencia de los clusters generados por KMeans.
- El uso de **PCA** facilitó la visualización en 2D, aunque a costa de perder interpretabilidad directa de los ejes.
- KMeans fue capaz de segmentar de forma eficiente a los usuarios, especialmente tras eliminar las anomalías más evidentes.

---

Este enfoque híbrido entre clustering y detección de anomalías proporciona un pipeline robusto para detectar fraudes o transacciones atípicas en sistemas financieros.


In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Selección de variables numéricas
num_cols = df.select_dtypes(include=[np.number]).columns

# Escalar los datos (KMeans es sensible a las magnitudes)
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[num_cols])

# Elegimos 3 Clusters como ejemplo
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster'] = kmeans.fit_predict(df_scaled)

# Revisar la cantidad de registros por cluster
print(df['cluster'].value_counts())

# Visualización de los Clusters
sns.scatterplot(x='Transaction_amount', y='Average Amount/transaction/day', hue='cluster', data=df)
plt.title('Clustering con KMeans')
plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df[['Transaction_amount', 'Average Amount/transaction/day']])

# Analisis comparativa con el uso de ELBOW METHOD metrics


Para determinar el número adecuado de clusters al aplicar KMeans, se utilizó el método del codo (Elbow Method), que evalúa la inercia (suma de las distancias cuadradas internas dentro de los clusters) para distintos valores de k.

Observaciones del Gráfico
En el gráfico se observa una disminución pronunciada de la inercia hasta k = 3, momento en el que la curva empieza a aplanarse.

Este "codo" en la curva sugiere que k = 3 es una buena elección, ya que incrementar el número de clusters más allá de ese punto no reduce significativamente la inercia.

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

distortions = []
K = range(1, 10)
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(df_scaled)
    distortions.append(kmeans.inertia_)

plt.plot(K, distortions, 'bx-')
plt.xlabel('Número de Clusters k')
plt.ylabel('Inercia')
plt.title('Elbow Method para determinar k')
plt.show()

### Ahora podemos ver lo siguiente:

## Cluster	Comportamiento Detectado
0.	Clientes con bajo consumo y transacciones bajas.
1.	Clientes con consumo medio.
2.	Clientes con alto consumo. Son los VIP o anómalos.

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
df['cluster'] = kmeans.fit_predict(df_scaled)

sns.scatterplot(x='Transaction_amount', y='Average Amount/transaction/day', hue='cluster', data=df)
plt.title('Nuevo Clustering con KMeans Sin PCA')
plt.show()

# Analisis aplicación de cluster sobre las características originales, sin reducción de dimensionalidad (PCA).


Transaction_amount (Monto total de transacciones)

Average Amount/transaction/day (Promedio diario por transacción)

Descripción:

Se aplicó KMeans directamente sobre las características originales, sin reducción de dimensionalidad (PCA).

+ El modelo agrupó los datos en 3 clusters:

+ Cluster 0 (negro): transacciones de bajo monto y baja frecuencia diaria.

+ Cluster 1 (rojo oscuro): montos y frecuencias intermedias.

+ Cluster 2 (rosado claro): incluye las transacciones más altas y también las más esporádicas.

Análisis:

El cluster más claro (rosado claro) contiene valores extremos, lo cual podría indicar anomalías o transacciones atípicas (potencialmente fraudulentas).
El clustering sin PCA ofrece una visión más directa del comportamiento financiero basado en las métricas reales.
Aunque KMeans no está diseñado para detectar outliers, los puntos dispersos en el cluster 2 sugieren una mayor varianza.

Ventajas:

Resultados más interpretables en función de los valores reales.
Útil para analizar directamente el impacto del monto y frecuencia sin transformar los datos.

Limitaciones:

KMeans es sensible a la escala de los datos y a outliers.
Supone que los clusters son esféricos y de tamaño similar, lo cual puede no reflejar la estructura real de los datos.

In [ ]:
# @title pruebas realizadas con StandardScaler y KMeans con metrics silhouette_score

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Cargar datos
df2 = pd.read_csv('/content/creditcardcsvpresent.csv')

# Análisis exploratorio
print(df2.info())
print(df2.describe())

# Selección de variables numéricas
num_cols = ['Transaction_amount', 'Average Amount/transaction/day']
X = df2[num_cols]

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Clustering
kmeans = KMeans(n_clusters=3, random_state=42)
df2['cluster'] = kmeans.fit_predict(X_scaled)

# Visualización
plt.figure(figsize=(8,6))
sns.scatterplot(data=df2, x='Transaction_amount', y='Average Amount/transaction/day', hue='cluster', palette='rocket')
plt.title('Clustering con KMeans Sin PCA')
plt.show()


In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

# Escalado
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[num_cols])

# DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
clusters = dbscan.fit_predict(X_scaled)

df2['cluster'] = clusters


sns.scatterplot(data=df, x="Transaction_amount", y="Average Amount/transaction/day", hue="cluster", palette="tab10")
plt.title("Clustering con DBSCAN Sin PCA")
plt.show()

In [ ]:
# Librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage

# Dataset de ejemplo (supongo que ya tienes tu DataFrame llamado df)
num_cols = ['Transaction_amount', 'Average Amount/transaction/day']

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[num_cols])

# Crear el modelo
agg_clustering = AgglomerativeClustering(n_clusters=3, linkage='ward')
clusters = agg_clustering.fit_predict(X_scaled)

# Agregar los clusters al DataFrame
df['cluster'] = clusters



# Clustering Jerárquico - Agglomerative Clustering Sin PCA
Transaction_amount (Monto total de transacciones)
Average Amount/transaction/day (Promedio diario por transacción)
## Observaciones:

DBSCAN identificó 3 clusters principales.

Es capaz de detectar grupos con forma arbitraria y distinguir outliers (aunque no se visualizan explícitamente en negro, podrían estar filtrados).

El grupo verde (Cluster 2) incluye valores extremos, posiblemente relacionados con transacciones atípicas o fraudulentas.

Los otros dos grupos (azul y naranja) representan clientes con patrones de gasto más regulares.

Ventaja: no necesita definir el número de clusters y es robusto frente a outliers.

Limitación: sensible a la elección de eps y min_samples.

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(data=df, x='Transaction_amount', y='Average Amount/transaction/day', hue='cluster', palette='tab10')
plt.title("Clustering Jerárquico - Agglomerative Clustering Sin PCA")
plt.show()


# Dendrograma jerárquico que representa la distancia entre las muestras.

Observaciones:
Muestra cómo se forman los clusters mediante fusiones sucesivas.

En este dendrograma, un corte a cierta altura (por ejemplo, en el eje Y cerca de 60) sugiere una división en dos clusters grandes.
Ideal para analizar la estructura de los datos, aunque poco escalable para grandes volúmenes.
Ventaja: no requiere definir el número de clusters de entrada.


Se realiza de manera representativa la definición de outliers

In [ ]:
linked = linkage(X_scaled, method='ward')

plt.figure(figsize=(12, 6))
dendrogram(linked,
           orientation='top',
           distance_sort='descending',
           show_leaf_counts=False)
plt.title('Dendrograma - Agglomerative Clustering')
plt.xlabel('Muestras')
plt.ylabel('Distancia')
plt.show()


Características usadas:
Datos reducidos mediante PCA (Componentes Principales) → proyección en 2D.
+ KMeans Clustering
+ DBSCAN Clustering
+ Agglomerative Clustering
Observaciones:

Se agruparon los datos en 3 clusters.

La proyección PCA muestra dos grupos claramente densos (rojo y azul) y un grupo más disperso (verde).

El grupo verde (Cluster 2) probablemente representa comportamientos de gasto más variables o inusuales.

Este método es eficiente computacionalmente pero sensible a la escala y a outliers.

Ventaja: rápido y fácil de implementar.

Limitación: supone que los clusters son esféricos y de tamaño similar.

In [ ]:
# Librerías necesarias
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el dataset
df = pd.read_csv('/content/creditcardcsvpresent.csv')  # Cambia el path si es necesario

# Eliminamos columnas con muchos nulos
df = df.drop(columns=['Transaction date'])

# Selección de variables numéricas
num_cols = df.select_dtypes(include=[np.number]).columns

# Escalado de datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df[num_cols])

# Reducción de dimensionalidad a 2D
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

# Clustering con KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
df['kmeans_cluster'] = kmeans.fit_predict(X_scaled)

# Clustering con DBSCAN
dbscan = DBSCAN(eps=2, min_samples=5)
df['dbscan_cluster'] = dbscan.fit_predict(X_scaled)

# Clustering con Agglomerative Clustering
agg = AgglomerativeClustering(n_clusters=3)
df['agg_cluster'] = agg.fit_predict(X_scaled)

# Función para graficar clusters
def plot_clusters(X, labels, title):
    plt.figure(figsize=(6,4))
    sns.scatterplot(x=X[:,0], y=X[:,1], hue=labels, palette='Set1', legend='full')
    plt.title(title)
    plt.show()

# Gráficas
plot_clusters(X_pca, df['kmeans_cluster'], 'KMeans Clustering')
plot_clusters(X_pca, df['dbscan_cluster'], 'DBSCAN Clustering')
plot_clusters(X_pca, df['agg_cluster'], 'Agglomerative Clustering')


# KMeans Clustering - Reducción de dimensionalidad con PCA

## Análisis de Componentes Principales

Visualización: Proyección bidimensional (2D) de los datos transformados por PCA.

📊 Observaciones del Gráfico
Se identifican tres clústeres distintos, visualmente separados:

Cluster 0 (rojo): Agrupa una gran parte de las observaciones centradas alrededor del origen con forma de triángulo invertido.

Cluster 1 (azul): A la izquierda del rojo, con una orientación algo más vertical.

Cluster 2 (verde): Abarca un espacio más amplio, incluyendo puntos dispersos, lo que sugiere mayor variabilidad o presencia de outliers.

💡 Ventajas del uso de PCA
Reducción de ruido y dimensiones permite visualizar de manera clara las separaciones de los grupos.

Facilita la interpretación visual del agrupamiento.

⚠️ Limitaciones
Pérdida de interpretabilidad directa: Las nuevas dimensiones (componentes principales) no representan atributos originales como transaction_amount o avg_amount_per_day, dificultando la explicación semántica del agrupamiento.

Outliers visibles: El cluster 2 incluye puntos alejados del centro, lo cual puede afectar la precisión del modelo si no se gestionan adecuadamente.

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Cargar tus datos
df = pd.read_csv('/content/creditcardcsvpresent.csv')

# 2. Seleccionar las variables numéricas que te interesan
X = df[['Transaction_amount', 'Average Amount/transaction/day']]  # Change this line

# 3. Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Aplicar PCA
pca = PCA(n_components=2)  # Reducimos a 2 dimensiones para graficar
X_pca = pca.fit_transform(X_scaled)

# 5. Aplicar Clustering
kmeans = KMeans(n_clusters=3)
clusters_kmeans = kmeans.fit_predict(X_pca)

agg = AgglomerativeClustering(n_clusters=3)
clusters_agg = agg.fit_predict(X_pca)

# 6. Graficar KMeans
plt.figure(figsize=(6,5))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=clusters_kmeans, palette='Set1')
plt.title('Clustering con KMeans (PCA)')
plt.show()

# 7. Graficar Agglomerative
plt.figure(figsize=(6,5))
sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=clusters_agg, palette='Set2')
plt.title('Clustering Jerárquico - Agglomerative (PCA)')
plt.show()


---

### Objetivo de Uso del Dataset en este Laboratorio

Aplicar técnicas de aprendizaje no supervisado (K-Means Clustering) para explorar posibles agrupaciones de las transacciones y analizar si existen patrones de comportamiento diferenciados entre transacciones legítimas y fraudulentas.

Esto permitirá evaluar la utilidad del clustering como técnica exploratoria dentro de un sistema de análisis de fraudes.

---

Mediante este trabajo se pretende que pongas en práctica la aplicación de los algoritmos de detección de anomalías u outliers y las técnicas de agrupamiento. El objetivo es que comprendas de forma práctica con un problema determinado los pasos que hay que realizar para detección automática de valores inusuales y, por otro lado, analizar los clúster o grupos resultado de aplicar un algoritmo de agrupamiento.

| **Técnica**                  | **Entrada**          | **Visualización** | **Observaciones clave**                        |
| ---------------------------- | -------------------- | ----------------- | ---------------------------------------------- |
| **DBSCAN Sin PCA**           | Variables originales | 2D                | Detecta formas arbitrarias y outliers.         |
| **KMeans Con PCA**           | Datos reducidos      | PCA 2D            | Rápido, pero pierde interpretabilidad directa. |
| **Agglomerative Clustering** | Variables originales | Dendrograma       | Muestra jerarquía de agrupación.               |
| **KMeans Sin PCA**           | Variables originales | 2D                | Más interpretable, pero sensible a outliers.   |